<a href="https://colab.research.google.com/github/audalsgh/20250624/blob/main/0625_python_%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

샘플1. 핸들 각도를 조절하는 함수 정의

In [26]:
def handle_right_turn():
  angle += 10
def handle_left_turn():
  angle -= 10

샘플2. 좌측, 우측 차선이 이탈되는게 감지되면, 반대로 핸들을 꺾는 함수를 호출한다

In [27]:
def left_line_detected(detecting):
  if (detecting == True):
    handle_right_turn()

def right_line_detected(detecting):
  if (detecting == True):
    handle_left_turn()

샘플3. 가속, 감속 함수 정의.

In [28]:
speed = 0

def acceleration():
  global speed
  speed += 1
def breaking():
  global speed
  speed -= 1

샘플4. 라이다 센서로 장애물이 가까운 위치라고 판단되면, 감속 함수 호출.

In [29]:
def obstacle_dodge(lidar_data):
  if (lidar_data < 10):
    print("장애물과 너무 가깝습니다. 급정거합니다")
    breaking()
  elif (lidar_data < 30):
    print("장애물이 발견되었습니다. 주의하세요")

샘플5. 신호등 색을 감지하여, 출발과 정지를 결정하는 함수 정의.

In [30]:
def traffic_lights_detect(cam_data):
  if (cam_data == "빨간불"):
    print("빨간불이니 정지합니다")
    for i in range(10):
      breaking()
  if (cam_data == "초록불"):
    print("초록불이니 출발합니다")
    for i in range(10):
      acceleration()

샘플6. 속도를 입력받아 과속을 검사하는 if문, 클래스 코드 샘플

In [31]:
class SpeedMonitor:
    def __init__(self, speed_limit=60):
        self.speed_limit = speed_limit

    def check_speed(self, current_speed):
        if (current_speed > self.speed_limit):
            return "과속중입니다. 속도를 줄입니다"
        elif (current_speed == self.speed_limit):
            return "제한 속도입니다. 속도를 줄이세요"
        else:
            return "안전 속도입니다."

if __name__ == "__main__":
    monitor = SpeedMonitor()  # SpeedMonitor 클래스의 객체인 monior라는 변수 정의.
    current_speed = int(input("현재 속도를 입력하세요 (km/h): "))
    message = monitor.check_speed(current_speed)
    print(message)

현재 속도를 입력하세요 (km/h): 80
과속중입니다. 속도를 줄입니다


샘플7. vehicle 클래스는 속도 데이터를 갖는다. car1 객체를 만들어 속도가 변하도록 했다

In [32]:
class vehicle:
  def __init__(self, speed=0):
    self.speed = speed

lidar_data = 30
cam_data = "초록불"

car1 = vehicle()
car1.speed = obstacle_dodge(lidar_data)
car1.speed = traffic_lights_detect(cam_data)

초록불이니 출발합니다


샘플8. 주행중 차선 중앙에 맞춰지도록 하는 클래스 샘플

In [33]:
class LaneKeeping:
    def __init__(self, center_threshold=0.1):
        # 편차가 ±0.1 이내면 중앙 유지
        self.threshold = center_threshold
        self.offset = 0.0

    def update_offset(self, offset):
        # offset >0: 오른쪽으로 치우침, <0: 왼쪽으로 치우침
        self.offset = offset
        if self.offset > self.threshold:
            print("좌측 조향: 차선 중앙으로 이동")
        elif self.offset < -self.threshold:
            print("우측 조향: 차선 중앙으로 이동")
        else:
            print("직진 유지: 차선 중앙에 위치")

lk = LaneKeeping()
lk.update_offset(0.15)   # 오른쪽 치우침 → 좌측 조향
lk.update_offset(-0.2)   # 왼쪽 치우침 → 우측 조향
lk.update_offset(0.05)   # 중앙 유지

좌측 조향: 차선 중앙으로 이동
우측 조향: 차선 중앙으로 이동
직진 유지: 차선 중앙에 위치


샘플9. 장애물 감지 시스템을 클래스로 구현해보기.

In [34]:
class ObstacleAvoidance:
    def __init__(self, safe_distance=20):
        # cm 단위 안전 거리
        self.safe_distance = safe_distance

    def analyze(self, front_dist, left_dist, right_dist):
        # front_dist: 전방 장애물 거리
        if front_dist < self.safe_distance:
            # 전방 막히면 좌우 중 더 넓은 쪽으로 회피
            if left_dist > right_dist:
                print("좌측 회피: 왼쪽으로 우회합니다")
            else:
                print("우측 회피: 오른쪽으로 우회합니다")
        else:
            print("전방 클리어: 직진")

oa = ObstacleAvoidance()
oa.analyze(front_dist=15, left_dist=30, right_dist=10)  # 전방 가까움 → 좌측 회피
oa.analyze(front_dist=25, left_dist=5,  right_dist=5)   # 전방 클리어 → 직진

좌측 회피: 왼쪽으로 우회합니다
전방 클리어: 직진


샘플10. 미리 정의한 웨이포인트 목록을 관리하며, 다음 웨이포인트까지 진행상태를 출력하는 클래스 샘플

In [35]:
class RoutePlanner:
    def __init__(self, waypoints):
        # waypoints: [(x1,y1), (x2,y2), ...]
        self.waypoints = waypoints
        self.current_index = 0

    def get_current(self):
        return self.waypoints[self.current_index]

    def advance(self):
        if self.current_index < len(self.waypoints) - 1:
            self.current_index += 1
            print(f"다음 웨이포인트로 이동: {self.get_current()}")
        else:
            print("최종 목적지에 도달했습니다")

wps = [(0,0), (10,5), (20,15)]
rp = RoutePlanner(wps)
print("현재 목표:", rp.get_current())  # (0,0)
rp.advance()                           # (10,5)
rp.advance()                           # (20,15)
rp.advance()                           # 최종 도달

현재 목표: (0, 0)
다음 웨이포인트로 이동: (10, 5)
다음 웨이포인트로 이동: (20, 15)
최종 목적지에 도달했습니다
